In [ ]:
import requests
import os
import itertools
import pandas as pd

# API endpoint
url = "https://www.unitedforalice.org/api/survival-budget-calculator"

# Base parameters for the API request
params_base = {
    'sort': '',
    'group': '',
    'filter': '',
    'stateFP': '18',          # Indiana state
    'year': '2022',           # Year 
}

# Mapping of county FIPS codes to county names (all Indiana counties)
county_names = {
    '001': 'Adams',
    '003': 'Allen',
    '005': 'Bartholomew',
    '007': 'Benton',
    '009': 'Blackford',
    '011': 'Boone',
    '013': 'Brown',
    '015': 'Carroll',
    '017': 'Cass',
    '019': 'Clark',
    '021': 'Clay',
    '023': 'Clinton',
    '025': 'Crawford',
    '027': 'Daviess',
    '029': 'Dearborn',
    '031': 'Decatur',
    '033': 'DeKalb',
    '035': 'Delaware',
    '037': 'Dubois',
    '039': 'Elkhart',
    '041': 'Fayette',
    '043': 'Floyd',
    '045': 'Fountain',
    '047': 'Franklin',
    '049': 'Fulton',
    '051': 'Gibson',
    '053': 'Grant',
    '055': 'Greene',
    '057': 'Hamilton',
    '059': 'Hancock',
    '061': 'Harrison',
    '063': 'Hendricks',
    '065': 'Henry',
    '067': 'Howard',
    '069': 'Huntington',
    '071': 'Jackson',
    '073': 'Jasper',
    '075': 'Jay',
    '077': 'Jefferson',
    '079': 'Jennings',
    '081': 'Johnson',
    '083': 'Knox',
    '085': 'Kosciusko',
    '087': 'LaGrange',
    '089': 'Lake',
    '091': 'LaPorte',
    '093': 'Lawrence',
    '095': 'Madison',
    '097': 'Marion',
    '099': 'Marshall',
    '101': 'Martin',
    '103': 'Miami',
    '105': 'Monroe',
    '107': 'Montgomery',
    '109': 'Morgan',
    '111': 'Newton',
    '113': 'Noble',
    '115': 'Ohio',
    '117': 'Orange',
    '119': 'Owen',
    '121': 'Parke',
    '123': 'Perry',
    '125': 'Pike',
    '127': 'Porter',
    '129': 'Posey',
    '131': 'Pulaski',
    '133': 'Putnam',
    '135': 'Randolph',
    '137': 'Ripley',
    '139': 'Rush',
    '141': 'St. Joseph',
    '143': 'Scott',
    '145': 'Shelby',
    '147': 'Spencer',
    '149': 'Starke',
    '151': 'Steuben',
    '153': 'Sullivan',
    '155': 'Switzerland',
    '157': 'Tippecanoe',
    '159': 'Tipton',
    '161': 'Union',
    '163': 'Vanderburgh',
    '165': 'Vermillion',
    '167': 'Vigo',
    '169': 'Wabash',
    '171': 'Warren',
    '173': 'Warrick',
    '175': 'Washington',
    '177': 'Wayne',
    '179': 'Wells',
    '181': 'White',
    '183': 'Whitley'
}

# Range of options (0 to 6)
options_range = range(7)

# All possible combinations of demographic options
combinations = list(itertools.product(options_range, repeat=5))

# Folder path to store the final combined CSV
folder_path = r'C:\Users\mouny\Downloads\indiana_budget'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# List to hold the rows of the final dataframe
data_rows = []

# Loop through each county
for countyFP, county_name in county_names.items():
    print(f"Processing county: {county_name}")
    
    # Update countyFP in params_base
    params_base['countyFP'] = countyFP
    
    # Loop through each combination of demographics
    for combination in combinations:
        adults, seniors, school_age, pre_schoolers, infants = combination

        # Applying constraints to limit total adults and children to a maximum of 6 each
        total_adults = adults + seniors
        total_children = school_age + pre_schoolers + infants

        if total_adults <= 6 and total_children <= 6:
            # Prepare request parameters with the current combination
            params = params_base.copy()
            params.update({
                'adults': str(adults),
                'seniors': str(seniors),
                'schoolAge': str(school_age),
                'preSchoolers': str(pre_schoolers),
                'infants': str(infants)
            })

            # Make the API request
            response = requests.get(url, params=params)

            if response.status_code == 200:
                data = response.json()  # Parse the response JSON

                # If data is available for the current combination
                if data['Data']:
                    # Extract the necessary budget categories from the response
                    budget = {
                        'Housing - Rent': None,
                        'Housing - Utilities': None,
                        'Child Care': None,
                        'Food': None,
                        'Transportation': None,
                        'Health Care': None,
                        'Technology': None,
                        'Miscellaneous': None,
                        'Tax Payments': None,
                        'Tax Credits': None,
                        'Monthly Total': None,
                        'ANNUAL TOTAL': None,
                        'Hourly Wage': None
                    }

                    # Populate the budget dictionary with the API response data
                    for item in data['Data']:
                        if item['Name'] in budget:
                            budget[item['Name']] = item['Costs']

                    # Add the row with demographics and budget data
                    data_rows.append({
                        'State': 'Indiana',
                        'County': county_name,
                        'Adults': adults,
                        'Seniors': seniors,
                        'SchoolAge': school_age,
                        'PreSchoolers': pre_schoolers,
                        'Infants': infants,
                        **budget
                    })
                    print(f"Data for {county_name} (Adults: {adults}, Seniors: {seniors}, SchoolAge: {school_age}, PreSchoolers: {pre_schoolers}, Infants: {infants}) processed.")
                else:
                    print(f"No data for {county_name} (Adults: {adults}, Seniors: {seniors}, SchoolAge: {school_age}, PreSchoolers: {pre_schoolers}, Infants: {infants})")
            else:
                print(f"Error for {county_name}: {response.status_code} - {response.text}")
        else:
            print(f"Skipped combination for {county_name}: (Adults: {adults}, Seniors: {seniors}, SchoolAge: {school_age}, PreSchoolers: {pre_schoolers}, Infants: {infants}) due to constraints.")

# Creating a DataFrame with all the collected data
columns = [
    'State', 'County', 'Adults', 'Seniors', 'SchoolAge', 'PreSchoolers', 'Infants',
    'Housing - Rent', 'Housing - Utilities', 'Child Care', 'Food', 'Transportation',
    'Health Care', 'Technology', 'Miscellaneous', 'Tax Payments', 'Tax Credits',
    'Monthly Total', 'ANNUAL TOTAL', 'Hourly Wage'
]
combined_df = pd.DataFrame(data_rows, columns=columns)

# Save the combined data into a single CSV file
combined_file_path = os.path.join(folder_path, 'indiana_combined_data.csv')
combined_df.to_csv(combined_file_path, index=False)

print(f"All data combined into {combined_file_path}")